In [6]:
print("hi")

hi


In [7]:
import pandas as pd

# Load global AQI data
df_global = pd.read_csv("global_air_pollution_dataset.csv")

# Filter Indian cities only
df_india = df_global[df_global["Country"] == "India"]

# Drop rows with missing AQI
df_india = df_india.dropna(subset=["AQI Value"])

# Drop duplicate cities (keep first)
df_india = df_india.drop_duplicates(subset=["City"])

# Reset index
df_india.reset_index(drop=True, inplace=True)
df_india.head()


,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category
0,India,Radaur,158,Unhealthy,3,Good,139,Unhealthy for Sensitive Groups,1,Good,158,Unhealthy
1,India,Rajgir,154,Unhealthy,3,Good,100,Unhealthy for Sensitive Groups,2,Good,154,Unhealthy
2,India,Phulabani,161,Unhealthy,2,Good,71,Moderate,0,Good,161,Unhealthy
3,India,Pimpri,118,Unhealthy for Sensitive Groups,2,Good,30,Good,2,Good,118,Unhealthy for Sensitive Groups
4,India,Piravam,81,Moderate,1,Good,24,Good,1,Good,81,Moderate


In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Define features and target
features = ["CO AQI Value", "Ozone AQI Value", "NO2 AQI Value", "PM2.5 AQI Value"]
X = df_india[features]
y = df_india["AQI Value"]

# Split for training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate
y_pred = rf.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))


MSE: 44.92680963855421


In [12]:
geo_df = pd.read_csv("geo_locations_indian_cities.csv")
geo_df = geo_df.rename(columns={"ASCII Name": "City", "Latitude": "Latitude", "Longitude": "Longitude"})
geo_df = geo_df[["City", "Latitude", "Longitude"]].drop_duplicates()
geo_df.reset_index(drop=True, inplace=True)
geo_df.head()


,City,Latitude,Longitude
0,Ziro,27.59497,93.83854
1,Wardha,20.73933,78.59784
2,Vijapur,23.56230,72.74848
3,Vettaikkaranpudur,10.56207,76.91305
4,Vadnagar,23.78593,72.63893


In [16]:
df_geo = pd.read_csv("IN.txt", sep='\t', header=0)

# Check what columns are there
df_geo.columns


C:\Users\jites\AppData\Local\Temp\ipykernel_20684\968519208.py:1: DtypeWarning: Columns (9,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geo = pd.read_csv("IN.txt", sep='\t', header=0)


Index(['1114940', 'Rāvi River', 'Ravi River',
       'Ravi,Ravi River,Rāvi,Rāvi River', '30.62123', '71.82683', 'H', 'STM',
       'IN', 'IN,PK', '00', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', '0',
       'Unnamed: 15', '133', 'Asia/Kolkata', '2023-11-07'],
      dtype='object')

In [17]:
colnames = [
    'Geoname ID', 'Name', 'ASCII Name', 'Alternate Names', 'Latitude',
    'Longitude', 'Feature Class', 'Feature Code', 'Country Code', 'CC2',
    'Admin1 Code', 'Admin2 Code', 'Admin3 Code', 'Admin4 Code',
    'Population', 'Elevation', 'DEM', 'Timezone', 'Modification Date'
]

df_geo = pd.read_csv("IN.txt", sep='\t', header=None, names=colnames)

# Now extract city names and coordinates
df_geo_clean = df_geo[['ASCII Name', 'Latitude', 'Longitude']].copy()
df_geo_clean.rename(columns={'ASCII Name': 'City'}, inplace=True)
df_geo_clean['City'] = df_geo_clean['City'].str.lower()


C:\Users\jites\AppData\Local\Temp\ipykernel_20684\3681623150.py:8: DtypeWarning: Columns (9,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geo = pd.read_csv("IN.txt", sep='\t', header=None, names=colnames)


In [20]:
air_df = pd.read_csv("global_air_pollution_dataset.csv")


In [21]:
indian_aqi_df = air_df[air_df['Country'] == 'India'].copy()
indian_aqi_df['City'] = indian_aqi_df['City'].str.lower()


In [22]:
colnames = [
    'Geoname ID', 'Name', 'ASCII Name', 'Alternate Names', 'Latitude',
    'Longitude', 'Feature Class', 'Feature Code', 'Country Code', 'CC2',
    'Admin1 Code', 'Admin2 Code', 'Admin3 Code', 'Admin4 Code',
    'Population', 'Elevation', 'DEM', 'Timezone', 'Modification Date'
]

df_geo = pd.read_csv("IN.txt", sep='\t', header=None, names=colnames)
df_geo_clean = df_geo[['ASCII Name', 'Latitude', 'Longitude']].copy()
df_geo_clean.rename(columns={'ASCII Name': 'City'}, inplace=True)
df_geo_clean['City'] = df_geo_clean['City'].str.lower()


C:\Users\jites\AppData\Local\Temp\ipykernel_20684\947637384.py:8: DtypeWarning: Columns (9,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geo = pd.read_csv("IN.txt", sep='\t', header=None, names=colnames)


In [23]:
merged_df = pd.merge(df_geo_clean, indian_aqi_df, on='City', how='left')


In [24]:
missing_aqi = merged_df['AQI Value'].isna().sum()
print(f"Cities with missing AQI: {missing_aqi}")


Cities with missing AQI: 629704


In [25]:
df_geo_filtered = df_geo[df_geo['Population'] > 10000].copy()
df_geo_filtered = df_geo_filtered[['ASCII Name', 'Latitude', 'Longitude']]
df_geo_filtered = df_geo_filtered.rename(columns={'ASCII Name': 'City'})
df_geo_filtered['City'] = df_geo_filtered['City'].str.lower()


In [27]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Cities with AQI (training data)
train_df = merged_df.dropna(subset=['AQI Value']).copy()
X_train = train_df[['Latitude', 'Longitude']]
y_train = train_df['AQI Value']


In [28]:
# Cities without AQI
predict_df = merged_df[merged_df['AQI Value'].isna()].copy()
X_predict = predict_df[['Latitude', 'Longitude']]


In [29]:
knn = KNeighborsRegressor(n_neighbors=5)  # you can adjust k
knn.fit(X_train, y_train)

predicted_aqi = knn.predict(X_predict)


In [30]:
predict_df['AQI Value'] = predicted_aqi

# Merge back with full dataset
final_df = pd.concat([train_df, predict_df], ignore_index=True)


In [31]:
final_df.to_csv("predicted_aqi_all_cities.csv", index=False)


In [6]:
import streamlit as st
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from streamlit_folium import st_folium
import plotly.express as px



@st.cache_data
def load_data():
    aqi_df = pd.read_csv("predicted_aqi_all_cities.csv")
    geo_df = pd.read_csv("geo_locations_indian_cities.csv")

    aqi_df['City'] = aqi_df['City'].str.strip().str.title()
    geo_df['Name'] = geo_df['Name'].str.strip().str.title()

    merged_df = pd.merge(aqi_df, geo_df, left_on='City', right_on='Name', how='inner')
    merged_df = merged_df.rename(columns={'Latitude_x': 'Latitude', 'Longitude_x': 'Longitude'})
    merged_df = merged_df.dropna(subset=['Latitude', 'Longitude', 'AQI Value'])

    return merged_df



2025-08-13 18:08:46.652 No runtime found, using MemoryCacheStorageManager


In [ ]:
df = load_data()
df.head()


,Latitude,Longitude,AQI Value,CO AQI Value,Ozone AQI Value,NO2 AQI Value,PM2.5 AQI Value,Geoname ID,Latitude_y,Longitude_y,Population,Digital Elevation Model
count,22324.000000,22324.000000,22324.000000,15178.000000,15178.000000,15178.000000,15178.000000,2.232400e+04,22324.000000,22324.000000,2.232400e+04,22324.000000
mean,24.372204,80.324922,179.104184,1.795362,62.953881,1.575702,176.646528,1.584913e+06,24.071916,80.032542,1.045841e+05,255.774503
std,4.921362,3.791730,92.407540,1.260298,44.861442,2.444583,94.455926,1.581375e+06,5.080251,4.781311,3.550455e+05,273.246179
min,8.018560,68.826550,23.000000,0.000000,1.000000,0.000000,13.000000,1.163626e+06,8.154420,68.826550,0.000000e+00,-9999.000000
25%,22.296510,77.688715,136.000000,1.000000,33.000000,0.000000,129.000000,1.260045e+06,21.901740,76.261010,1.220600e+04,97.000000
50%,25.557800,79.772490,168.000000,1.000000,45.000000,1.000000,167.000000,1.267939e+06,25.124460,79.263500,2.279600e+04,198.000000
75%,27.249407,82.778397,191.200000,3.000000,80.000000,2.000000,181.000000,1.273378e+06,27.172810,82.714420,5.674300e+04,329.000000
max,34.927400,96.810000,500.000000,12.000000,205.000000,43.000000,500.000000,1.217272e+07,34.927400,96.810000,1.269184e+07,3560.000000


In [10]:

df.describe()

,Latitude,Longitude,AQI Value,CO AQI Value,Ozone AQI Value,NO2 AQI Value,PM2.5 AQI Value,Geoname ID,Latitude_y,Longitude_y,Population,Digital Elevation Model
count,22324.000000,22324.000000,22324.000000,15178.000000,15178.000000,15178.000000,15178.000000,2.232400e+04,22324.000000,22324.000000,2.232400e+04,22324.000000
mean,24.372204,80.324922,179.104184,1.795362,62.953881,1.575702,176.646528,1.584913e+06,24.071916,80.032542,1.045841e+05,255.774503
std,4.921362,3.791730,92.407540,1.260298,44.861442,2.444583,94.455926,1.581375e+06,5.080251,4.781311,3.550455e+05,273.246179
min,8.018560,68.826550,23.000000,0.000000,1.000000,0.000000,13.000000,1.163626e+06,8.154420,68.826550,0.000000e+00,-9999.000000
25%,22.296510,77.688715,136.000000,1.000000,33.000000,0.000000,129.000000,1.260045e+06,21.901740,76.261010,1.220600e+04,97.000000
50%,25.557800,79.772490,168.000000,1.000000,45.000000,1.000000,167.000000,1.267939e+06,25.124460,79.263500,2.279600e+04,198.000000
75%,27.249407,82.778397,191.200000,3.000000,80.000000,2.000000,181.000000,1.273378e+06,27.172810,82.714420,5.674300e+04,329.000000
max,34.927400,96.810000,500.000000,12.000000,205.000000,43.000000,500.000000,1.217272e+07,34.927400,96.810000,1.269184e+07,3560.000000


In [11]:
df.columns

Index(['City', 'Latitude', 'Longitude', 'Country_x', 'AQI Value',
       'AQI Category', 'CO AQI Value', 'CO AQI Category', 'Ozone AQI Value',
       'Ozone AQI Category', 'NO2 AQI Value', 'NO2 AQI Category',
       'PM2.5 AQI Value', 'PM2.5 AQI Category', 'Geoname ID', 'Name',
       'ASCII Name', 'Alternate Names', 'Latitude_y', 'Longitude_y',
       'Population', 'Digital Elevation Model', 'Country_y', 'Coordinates'],
      dtype='object')

In [2]:
import pandas as pd

# Load big dataset
df = pd.read_csv("predicted_aqi_all_cities.csv", low_memory=False)

# Keep important columns
keep_cols = [
    'City', 'Latitude', 'Longitude', 'Country', 'AQI Value', 'AQI Category',
    'CO AQI Value', 'Ozone AQI Value', 'NO2 AQI Value', 'PM2.5 AQI Value'
]

df_small = df[keep_cols]
df_small = df_small[df_small['Country'] == 'India']
df_small = df_small.dropna(subset=['Latitude', 'Longitude', 'AQI Value'])

df_small.to_csv("aqi_india_pollutants.csv", index=False)
print("✅ Saved smaller dataset with pollutants:", len(df_small))


✅ Saved smaller dataset with pollutants: 27712
